# BLS price data

In [1]:
import altair as alt
import altair_stiles as altstiles
import pandas as pd
import numpy as np

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [2]:
%load_ext lab_black

---

## Get Codes

#### Read BLS' inflation and price codes

In [3]:
codes_src = pd.read_fwf("https://download.bls.gov/pub/time.series/ap/ap.series")

In [4]:
codes_src[
    [
        "area_code",
        "item_code",
        "series_title",
        "footnote_codes",
        "begin_year",
        "begin_period",
        "end_year",
        "end_period",
    ]
] = codes_src[
    "area_code\titem_code\tseries_title\tfootnote_codes\tbegin_year\tbegin_period\tend_year\tend_period"
].str.split(
    "\t", expand=True
)

In [5]:
codes_src = codes_src.drop(
    "area_code\titem_code\tseries_title\tfootnote_codes\tbegin_year\tbegin_period\tend_year\tend_period",
    axis=1,
)

#### Areas

In [6]:
area_codes = {
    "0000": "U.S. city average",
    "0100": "Northeast",
    "0110": "New England",
    "0120": "Middle Atlantic",
    "0200": "Midwest",
    "0230": "East North Central",
    "0240": "West North Central",
    "0300": "South",
    "0350": "South Atlantic",
    "0360": "East South Central",
    "0370": "West South Central",
    "0400": "West",
    "0480": "Mountain",
    "0490": "Pacific",
}

#### Just the codes for city averages

In [7]:
codes_df = codes_src[codes_src["area_code"] == "0000"].copy()

In [8]:
codes_df.head()

,series_id,area_code,item_code,series_title,footnote_codes,begin_year,begin_period,end_year,end_period
0,APU0000701111,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",,1980,M01,2022,M05
1,APU0000701311,0000,701311,"Rice, white, long grain, precooked (cost per p...",,1980,M01,1981,M12
2,APU0000701312,0000,701312,"Rice, white, long grain, uncooked, per lb. (45...",,1980,M01,2022,M05
3,APU0000701321,0000,701321,Spaghetti (cost per pound/453.6 grams) in U.S....,,1980,M01,1981,M03
4,APU0000701322,0000,701322,"Spaghetti and macaroni, per lb. (453.6 gm) in ...",,1984,M01,2022,M05


---

## Get data about food prices

#### Read BLS inflation and price measures

In [9]:
food_src = pd.read_fwf("https://download.bls.gov/pub/time.series/ap/ap.data.3.Food")

In [10]:
food_src.head()

,series_id,year,period,value,footnote_codes
0,APU0000701111,1980,M01,0.203,NaN
1,APU0000701111,1980,M02,0.205,NaN
2,APU0000701111,1980,M03,0.211,NaN
3,APU0000701111,1980,M04,0.206,NaN
4,APU0000701111,1980,M05,0.207,NaN


In [11]:
food_src.value = food_src.value.replace("-", np.nan).astype(float)

---

## Merge

#### Combine food price and code dataframes

In [12]:
merged_df = pd.merge(food_src, codes_df, on="series_id").drop(
    ["footnote_codes_x", "footnote_codes_y"], axis=1
)

In [13]:
merged_df["month"] = merged_df["period"].str[1:]

In [14]:
merged_df["date"] = pd.to_datetime(
    merged_df["year"].astype(str) + "-" + merged_df["month"] + "-" + "01"
)

In [15]:
for title in merged_df.series_title.unique():
    print(title)

Flour, white, all purpose, per lb. (453.6 gm) in U.S. city average, average price, not seasonally adjusted
Rice, white, long grain, precooked (cost per pound/453.6 grams) in U.S. city average, average price, not seasonally adjusted
Rice, white, long grain, uncooked, per lb. (453.6 gm) in U.S. city average, average price, not seasonally adjusted
Spaghetti (cost per pound/453.6 grams) in U.S. city average, average price, not seasonally adjusted
Spaghetti and macaroni, per lb. (453.6 gm) in U.S. city average, average price, not seasonally adjusted
Bread, white, pan, per lb. (453.6 gm) in U.S. city average, average price, not seasonally adjusted
Bread, French, per lb. (453.6 gm) in U.S. city average, average price, not seasonally adjusted
Bread, rye, pan (cost per pound/453.6 grams) in U.S. city average, average price, not seasonally adjusted
Bread, whole wheat, pan, per lb. (453.6 gm) in U.S. city average, average price, not seasonally adjusted
Bread, wheat blend, pan (cost per pound/453.

---

In [16]:
bacon = merged_df[merged_df["series_title"].str.contains("Bacon")]

In [17]:
bacon.dtypes

series_id               object
year                     int64
period                  object
value                  float64
area_code               object
item_code               object
series_title            object
begin_year              object
begin_period            object
end_year                object
end_period              object
month                   object
date            datetime64[ns]
dtype: object

In [18]:
alt.Chart(bacon,).mark_line().encode(
    x=alt.X("date", title="", axis=alt.Axis(tickCount=7, format="%Y")),
    y=alt.Y("value", axis=alt.Axis(format="$.2", tickCount=6), title=" "),
).properties(
    title="Average price of a pound of sliced bacon in U.S. cities",
    width=650,
    height=400,
)

alt.Chart(...)